# Calculating CpG ratio for the *Acropora millepora* transcriptome

This workflow calculates CpG ratio, or CpG O/E, for contigs in the *Acropora millepora* [transcriptome](http://www.ncbi.nlm.nih.gov/nuccore?term=74409%5BBioProject%5D). CpG ratio is an estimate of germline DNA methylation.

**NOTE: This particular workflow uses Genbank accession numbers for contig IDs, specifically for generating a CpG O/E file that can be joined with *A. millepora* gene expression data in Amil_expression.ipynb.**

This workflow is an extension of another IPython notebook workflow, `Amil_blast_anno.ipynb`, that generates an annotation of the same transcriptome. This workflow assumes that you have created the directories and files specified in the annotation workflow.

In [4]:
cd ../data/Amil

/Users/jd/Documents/Projects/Coral-CpG-ratio-MS/data/Amil


In [5]:
#fasta file
!head -2 Amil_Moya.fasta
!echo 
!echo number of seqs =
!fgrep -c ">" Amil_Moya.fasta

head: Amil_Moya.fasta: No such file or directory

number of seqs =
fgrep: Amil_Moya.fasta: No such file or directory


In [4]:
#Converting FASTA to tabular format and placing output file in analyses directory
!perl -e '$count=0; $len=0; while(<>) {s/\r?\n//; s/\t/ /g; if (s/^>//) { if ($. != 1) {print "\n"} s/ |$/\t/; $count++; $_ .= "\t";} else {s/ //g; $len += length($_)} print $_;} print "\n"; warn "\nConverted $count FASTA records in $. lines to tabular format\nTotal sequence length: $len\n\n";' \
Amil_Moya.fasta > ../../analyses/Amil/fasta2tab


Converted 52963 FASTA records in 1149658 lines to tabular format
Total sequence length: 71250280



In [7]:
cd ../../analyses/Amil

/Users/jd/Documents/Projects/Coral-CpG-ratio-MS/analyses/Amil


In [8]:
#Checking header on new tabular format file
!head -2 fasta2tab
!tail -2 fasta2tab

gi|379072745|gb|JR970414.1|	TSA: Acropora millepora Cluster034439.Acmimixed mRNA sequence	TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT
gi|379072746|gb|JR970415.1|	TSA: Acropora millepora Cluster034438.Acmimixed mRNA sequence	CGCCCCCACGTCGTCATCTGACGTTCCTGTCCTGTTGCTAAATCAGCCTATTGATTGCGGGAACACATCAATCAACTACTAAACAACAGAATCCTGGGTTTTCAGACTTACAGTGTCTCTGCGATGAGGAGAATGTTCCCCTGTCACCACAAGCACTCCTGTCTGGGCACTAAGACAAATCAGCAATGAGA

In [9]:
#Removing pipes
!tr '|' "\t" <fasta2tab> fasta2tab2
!head -2 fasta2tab2

gi	379072745	gb	JR970414.1		TSA: Acropora millepora Cluster034439.Acmimixed mRNA sequence	TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT
gi	379072746	gb	JR970415.1		TSA: Acropora millepora Cluster034438.Acmimixed mRNA sequence	CGCCCCCACGTCGTCATCTGACGTTCCTGTCCTGTTGCTAAATCAGCCTATTGATTGCGGGAACACATCAATCAACTACTAAACAACAGAATCCTGGGTTTTCAGACTTACAGTGTCTCTGCGATGAGGAGAATGTTCCCCTGTCACCACAAGCACTCCTGTCTGGGCACTAAGACAAATCAGCAATGAGA

In [8]:
#Add column with length of sequence
!perl -e '$col = 6;' -e 'while (<>) { s/\r?\n//; @F = split /\t/, $_; $len = length($F[$col]); print "$_\t$len\n" } warn "\nAdded column with length of column $col for $. lines.\n\n";' \
fasta2tab2 > tab_1


Added column with length of column 6 for 52963 lines.



In [10]:
!head -1 tab_1
!wc tab_1

gi	379072745	gb	JR970414.1		TSA: Acropora millepora Cluster034439.Acmimixed mRNA sequence	TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT	628
   52963  635556 76310959 tab_1


In [12]:
#Just printing contig ID in column 2
!awk '{print $4, "\t", $11, "\t", $12}' tab_1 > tab_2
!head -1 tab_2

JR970414.1 	 TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT 	 628


In [19]:
!sed 's/\S*\.1\S*//g' tab_2 > tab_3
!head tab_3

JR970414 	 TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT 	 628
JR970415 	 CGCCCCCACGTCGTCATCTGACGTTCCTGTCCTGTTGCTAAATCAGCCTATTGATTGCGGGAACACATCAATCAACTACTAAACAACAGAATCCTGGGTTTTCAGACTTACAGTGTCTCTGCGATGAGGAGAATGTTCCCCTGTCACCACAAGCACTCCTGTCTGGGCACTAAGACAAATCAGCAATGAGACATTCTTGGCTTCCAAATCAATAAGTGCACATTAACTGGTGTTTGGAGAGACCAATCACCTATCTAGATATGGTCCACCATATTGCAGATTGAAACAATGAATAATAGAACACAAACAATACCCTAACTTGACCACAATAGAAGGTACAGGT

In [20]:
#Instead of using the step to deal with description name issues above, the file used to count Cs and Gs will only include the sequence
!awk '{print $2}' tab_3 > tab_4

In [21]:
#This counts CGs - both cases
!echo "CG" | awk -F\[Cc][Gg] '{print NF-1}' tab_4 > CG 

In [22]:
#Counts Cs
!echo "C" | awk -F\[Cc] '{print NF-1}' tab_4 > C 

In [23]:
#Counts Gs
!echo "G" | awk -F\[Gg] '{print NF-1}' tab_4 > G 

In [24]:
#Combining counts
!paste tab_3 \
CG \
C \
G \
> comb
!head -5 comb

JR970414 	 TCATCATTATTTCTTTTTGTTTTTCTGTGATCTTCGGTCAATGCGATAGATCCTCGAGTTATCGTGACTGCGCTCCAGACCAAGTCTGTTGTAGGAGGCAGTGTTTTTACAGCTCGAACTGTTTATATCTGTCTTGCTCCATGGACTCCGATTGTTCAGTGAATGAAGTCTGTTGTAGCAGCAAGTGTCGTTCTGGTNNNNNNNNNNCTGACTGCAGTGGGGATTTTTGTCGCTCGAACAATGATTGCAGCGTTGGGCAAAAGTGTTGTGTGAATACCTGCACCAACTATGATTGTGAGGACCCTACCGTCGCCATTCTTATCGCGGTAGTGGGTTCGCTCGTGGGCTTATTTGTTGTTTTCATTTCAATTTACTACTGCCACAGAAGAGCTCGTTTGGGTCGTTCCGGTACAGCAGAGGTGGGAAGACAAGTTGCCCCAACCGATGCTATCACAACCCAATCAGCAAACCAACAAGGCTACGCATATCAGCAACTCCCTTAAATATCATCAGTATCAGACACCCATTTACAATCCGGAGACACAGAGACAACCAGGAGGAATACTTACTTCACATCGTGCATATGGTGAACTTCAAACCACTTGATCCCCATGTCCAGCGGAAGACT 	 628	25	145	140
JR970415 	 CGCCCCCACGTCGTCATCTGACGTTCCTGTCCTGTTGCTAAATCAGCCTATTGATTGCGGGAACACATCAATCAACTACTAAACAACAGAATCCTGGGTTTTCAGACTTACAGTGTCTCTGCGATGAGGAGAATGTTCCCCTGTCACCACAAGCACTCCTGTCTGGGCACTAAGACAAATCAGCAATGAGACATTCTTGGCTTCCAAATCAATAAGTGCACATTAACTGGTGTTTGGAGAGACCAATCACCTATCTAGATATGGTCCACCATATTGCAGATTGAAACAATGAATAATAGAACACAAACAATACCCTAACTTGACCACAATAG

# Calculating CpGo/e based on [Gavery and Roberts (2010)](http://www.biomedcentral.com/1471-2164/11/483)

<img src="http://eagle.fish.washington.edu/cnidarian/skitch/BMC_Genomics___Full_text___DNA_methylation_patterns_provide_insight_into_epigenetic_regulation_in_the_Pacific_oyster__Crassostrea_gigas__1A0683A5.png" alt="BMC_Genomics___Full_text___DNA_methylation_patterns_provide_insight_into_epigenetic_regulation_in_the_Pacific_oyster__Crassostrea_gigas__1A0683A5.png"/>

In [25]:
!awk '{print $1, "\t", (($4)/($5*$6))*(($3^2)/($3-1))}' comb > ID_CpG #use ^ instead of ** for exponent


In [26]:
!head ID_CpG2

JR970414 	 0.774633
JR970415 	 0.283791
JR970416 	 0.558113
JR970417 	 0.662023
JR970418 	 0.738617
JR970419 	 0.318018
JR970420 	 1.25641
JR970421 	 0.328632
JR970422 	 1.11106
JR970423 	 0.801276


# Now joining CpG to annotation, but first must sort files.

In [27]:
#Sorting CpG file
!sort ID_CpG2 > ID_CpG2.sorted
!head ID_CpG2.sorted

JR970414 	 0.774633
JR970415 	 0.283791
JR970416 	 0.558113
JR970417 	 0.662023
JR970418 	 0.738617
JR970419 	 0.318018
JR970420 	 1.25641
JR970421 	 0.328632
JR970422 	 1.11106
JR970423 	 0.801276
